In [ ]:
import os
import glob
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import datetime
import warnings
warnings.filterwarnings('ignore')

def extract_scenario_info(scenario_id):
    """Extract information from scenario ID"""
    # Pattern: SIM{n_sims}_N{sample_size}_F{n_features}_W{weight_type}
    pattern = r'SIM(\d+)_N(\d+)_F(\d+)_W(.+)'
    match = re.match(pattern, scenario_id)
    if match:
        n_sims, sample_size, n_features, weight_type = match.groups()
        return {
            'n_simulations': int(n_sims),
            'sample_size': int(sample_size),
            'n_features': int(n_features),
            'weight_type': weight_type
        }
    return None

def summarize_by_feature_count_and_weight(master_folder, output_folder="feature_weight_summaries"):
    """Summarize simulation results by feature count and weight distribution"""
    
    # Create output folder if it doesn't exist
    output_path = Path(output_folder)
    output_path.mkdir(exist_ok=True)
    
    print(f"Scanning {master_folder} for simulation results...")
    
    # Find all scenario folders
    all_scenarios = []
    for folder_path in Path(master_folder).glob("**/SIM*"):
        if folder_path.is_dir():
            scenario_id = folder_path.name
            scenario_info = extract_scenario_info(scenario_id)
            
            if scenario_info:
                scenario_info['folder_path'] = folder_path
                scenario_info['scenario_id'] = scenario_id
                all_scenarios.append(scenario_info)
    
    print(f"Found {len(all_scenarios)} valid scenario folders")
    
    # Group scenarios by feature count and weight type
    feature_weight_groups = {}
    for scenario in all_scenarios:
        n_features = scenario['n_features']
        weight_type = scenario['weight_type']
        key = (n_features, weight_type)
        
        if key not in feature_weight_groups:
            feature_weight_groups[key] = []
        feature_weight_groups[key].append(scenario)
    
    # Summary dataframes
    feature_importance_summary = []
    accuracy_summary = []
    meta_weights_summary = []
    correlation_summary = []  # New summary for rank correlation
    mad_summary = []  # New summary for mean absolute deviation
    
    # Process each feature count and weight type group
    for (n_features, weight_type), scenarios in sorted(feature_weight_groups.items()):
        print(f"\nProcessing {len(scenarios)} scenarios with {n_features} features and weight type {weight_type}...")
        
        # Initialize aggregated data
        all_importances = []
        all_accuracies = []
        all_meta_weights = []
        
        for scenario in scenarios:
            folder_path = scenario['folder_path']
            scenario_id = scenario['scenario_id']
            
            # Find feature importance CSV
            imp_files = list(folder_path.glob("*feature_importances.csv"))
            if imp_files:
                try:
                    # Read feature importance data
                    imp_df = pd.read_csv(imp_files[0])
                    
                    # Add scenario metadata
                    imp_df['scenario_id'] = scenario_id
                    imp_df['n_features'] = n_features
                    imp_df['weight_type'] = weight_type
                    imp_df['sample_size'] = scenario['sample_size']
                    imp_df['n_simulations'] = scenario['n_simulations']
                    
                    all_importances.append(imp_df)
                    
                    # Read accuracy data
                    acc_files = list(folder_path.glob("*accuracies.csv"))
                    if acc_files:
                        acc_df = pd.read_csv(acc_files[0])
                        acc_df['scenario_id'] = scenario_id
                        acc_df['n_features'] = n_features
                        acc_df['weight_type'] = weight_type
                        acc_df['sample_size'] = scenario['sample_size']
                        acc_df['n_simulations'] = scenario['n_simulations']
                        all_accuracies.append(acc_df)
                    
                    # Read meta weights data
                    meta_files = list(folder_path.glob("*meta_weights.csv"))
                    if meta_files:
                        meta_df = pd.read_csv(meta_files[0])
                        meta_df['scenario_id'] = scenario_id
                        meta_df['n_features'] = n_features
                        meta_df['weight_type'] = weight_type
                        meta_df['sample_size'] = scenario['sample_size']
                        meta_df['n_simulations'] = scenario['n_simulations']
                        all_meta_weights.append(meta_df)
                        
                except Exception as e:
                    print(f"Error processing {scenario_id}: {str(e)}")
        
        if all_importances:
            # Combine all data
            combined_importances = pd.concat(all_importances, ignore_index=True)
            combined_accuracies = pd.concat(all_accuracies, ignore_index=True) if all_accuracies else None
            combined_meta_weights = pd.concat(all_meta_weights, ignore_index=True) if all_meta_weights else None
            
            # Calculate feature importance summaries
            importance_stats = summarize_feature_importance(combined_importances, n_features, weight_type)
            feature_importance_summary.append(importance_stats)
            
            # Calculate accuracy summaries
            if combined_accuracies is not None:
                accuracy_stats = summarize_accuracies(combined_accuracies, n_features, weight_type)
                accuracy_summary.append(accuracy_stats)
            
            # Calculate meta-weight summaries
            if combined_meta_weights is not None:
                meta_stats = summarize_meta_weights(combined_meta_weights, n_features, weight_type)
                meta_weights_summary.append(meta_stats)
            
            # Calculate rank correlation and MAD with true weights
            if combined_importances is not None:
                corr_stats = calculate_rank_correlations(combined_importances, n_features, weight_type)
                correlation_summary.append(corr_stats)
                
                mad_stats = calculate_mean_absolute_deviations(combined_importances, n_features, weight_type)
                mad_summary.append(mad_stats)
            
            # Generate visualizations
            create_visualizations(combined_importances, combined_accuracies, 
                                combined_meta_weights, n_features, weight_type, output_path)
    
    # Combine all summaries
    if feature_importance_summary:
        fi_summary_df = pd.concat(feature_importance_summary, ignore_index=True)
        fi_summary_df.to_csv(output_path / "feature_importance_summary.csv", index=False)
        print(f"Feature importance summary saved to {output_path / 'feature_importance_summary.csv'}")
    
    if accuracy_summary:
        acc_summary_df = pd.concat(accuracy_summary, ignore_index=True)
        acc_summary_df.to_csv(output_path / "accuracy_summary.csv", index=False)
        print(f"Accuracy summary saved to {output_path / 'accuracy_summary.csv'}")
    
    if meta_weights_summary:
        meta_summary_df = pd.concat(meta_weights_summary, ignore_index=True)
        meta_summary_df.to_csv(output_path / "meta_weights_summary.csv", index=False)
        print(f"Meta weights summary saved to {output_path / 'meta_weights_summary.csv'}")
    
    if correlation_summary:
        corr_summary_df = pd.concat(correlation_summary, ignore_index=True)
        corr_summary_df.to_csv(output_path / "rank_correlation_summary.csv", index=False)
        print(f"Rank correlation summary saved to {output_path / 'rank_correlation_summary.csv'}")
    
    if mad_summary:
        mad_summary_df = pd.concat(mad_summary, ignore_index=True)
        mad_summary_df.to_csv(output_path / "mad_summary.csv", index=False)
        print(f"Mean absolute deviation summary saved to {output_path / 'mad_summary.csv'}")
    
    # Create overall comparison across feature counts and weight types
    if feature_importance_summary and accuracy_summary:
        create_feature_weight_comparison(fi_summary_df, acc_summary_df, 
                                     meta_summary_df if meta_weights_summary else None,
                                     corr_summary_df if correlation_summary else None,
                                     mad_summary_df if mad_summary else None,
                                     output_path)

def summarize_feature_importance(df, n_features, weight_type):
    """Calculate summary statistics for feature importance"""
    models = ['true_weights', 'xgboost', 'lgbm', 'random_forest', 
              'extra_trees', 'decision_tree', 'ada_boost', 'stacking']
    
    result_rows = []
    
    # For each model
    for model in models:
        model_cols = [col for col in df.columns if model in col and 'feature' in col]
        
        if not model_cols:
            continue
            
        # For each feature
        for feature_idx in range(1, n_features + 1):
            feature_col = f"{model}_feature_{feature_idx}"
            
            if feature_col not in df.columns:
                continue
                
            # Calculate statistics
            mean_value = df[feature_col].mean()
            median_value = df[feature_col].median()
            std_value = df[feature_col].std()
            var_value = df[feature_col].var()
            min_value = df[feature_col].min()
            max_value = df[feature_col].max()
            q1_value = df[feature_col].quantile(0.25)
            q3_value = df[feature_col].quantile(0.75)
            
            # Add row to results
            result_rows.append({
                'n_features': n_features,
                'weight_type': weight_type,
                'model': model,
                'feature': feature_idx,
                'mean': mean_value,
                'median': median_value,
                'std': std_value,
                'variance': var_value,
                'min': min_value,
                'max': max_value,
                'q1': q1_value,
                'q3': q3_value
            })
    
    # Create summary dataframe
    return pd.DataFrame(result_rows)

def summarize_accuracies(df, n_features, weight_type):
    """Calculate summary statistics for model accuracies"""
    result_rows = []
    
    # Calculate statistics for each model
    for model in df.columns:
        if model in ['scenario_id', 'n_features', 'weight_type', 'sample_size', 'n_simulations', 'Unnamed: 0']:
            continue
            
        # Calculate statistics
        mean_value = df[model].mean()
        median_value = df[model].median()
        std_value = df[model].std()
        var_value = df[model].var()
        min_value = df[model].min()
        max_value = df[model].max()
        q1_value = df[model].quantile(0.25)
        q3_value = df[model].quantile(0.75)
        
        # Add row to results
        result_rows.append({
            'n_features': n_features,
            'weight_type': weight_type,
            'model': model,
            'mean_accuracy': mean_value,
            'median_accuracy': median_value,
            'std_accuracy': std_value,
            'variance_accuracy': var_value,
            'min_accuracy': min_value,
            'max_accuracy': max_value,
            'q1_accuracy': q1_value,
            'q3_accuracy': q3_value
        })
    
    # Create summary dataframe
    return pd.DataFrame(result_rows)

def summarize_meta_weights(df, n_features, weight_type):
    """Calculate summary statistics for meta-model weights"""
    result_rows = []
    
    # Calculate statistics for each base model weight
    for col_idx in range(6):  # 6 base models
        if col_idx >= df.shape[1]:
            continue
            
        col = str(col_idx)
        if col not in df.columns:
            continue
            
        # Map column index to model name
        models = ['xgboost', 'lgbm', 'random_forest', 'extra_trees', 'decision_tree', 'ada_boost']
        model_name = models[col_idx] if col_idx < len(models) else f"model_{col_idx}"
        
        # Calculate statistics
        mean_value = df[col].mean()
        median_value = df[col].median()
        std_value = df[col].std()
        var_value = df[col].var()
        min_value = df[col].min()
        max_value = df[col].max()
        q1_value = df[col].quantile(0.25)
        q3_value = df[col].quantile(0.75)
        
        # Add row to results
        result_rows.append({
            'n_features': n_features,
            'weight_type': weight_type,
            'base_model': model_name,
            'mean_weight': mean_value,
            'median_weight': median_value,
            'std_weight': std_value,
            'variance_weight': var_value,
            'min_weight': min_value,
            'max_weight': max_value,
            'q1_weight': q1_value,
            'q3_weight': q3_value
        })
    
    # Create summary dataframe
    return pd.DataFrame(result_rows)

def calculate_rank_correlations(df, n_features, weight_type):
    """Calculate rank correlation between model feature importance and true weights using Kendall's Tau"""
    from scipy.stats import kendalltau
    
    models = ['xgboost', 'lgbm', 'random_forest', 
              'extra_trees', 'decision_tree', 'ada_boost', 'stacking']
    
    result_rows = []
    
    # Iterate through each simulation/row
    for idx in range(len(df)):
        row = df.iloc[idx]
        
        # Get true weights for this simulation
        true_weights = []
        for feature_idx in range(1, n_features + 1):
            true_col = f"true_weights_feature_{feature_idx}"
            if true_col in df.columns:
                true_weights.append(row[true_col])
        
        # Calculate Kendall's Tau for each model
        for model in models:
            model_weights = []
            for feature_idx in range(1, n_features + 1):
                model_col = f"{model}_feature_{feature_idx}"
                if model_col in df.columns:
                    model_weights.append(row[model_col])
            
            # Only calculate if we have data for both
            if true_weights and model_weights and len(true_weights) == len(model_weights):
                try:
                    # Calculate Kendall's Tau
                    tau, p_value = kendalltau(true_weights, model_weights)
                    
                    result_rows.append({
                        'n_features': n_features,
                        'weight_type': weight_type,
                        'model': model,
                        'sim_idx': idx,
                        'rank_correlation': tau,
                        'p_value': p_value
                    })
                except Exception as e:
                    print(f"Error calculating Kendall's Tau: {str(e)}")
    
    # Create summary dataframe
    corr_df = pd.DataFrame(result_rows)
    
    # Calculate summary statistics for Kendall's Tau
    summary_rows = []
    for model in models:
        model_corrs = corr_df[corr_df['model'] == model]
        
        if len(model_corrs) > 0:
            summary_rows.append({
                'n_features': n_features,
                'weight_type': weight_type,
                'model': model,
                'mean_rank_corr': model_corrs['rank_correlation'].mean(),
                'median_rank_corr': model_corrs['rank_correlation'].median(),
                'std_rank_corr': model_corrs['rank_correlation'].std(),
                'min_rank_corr': model_corrs['rank_correlation'].min(),
                'max_rank_corr': model_corrs['rank_correlation'].max()
            })
    
    return pd.DataFrame(summary_rows)

def calculate_mean_absolute_deviations(df, n_features, weight_type):
    """Calculate mean absolute deviation between model feature importance and true weights"""
    import numpy as np
    
    models = ['xgboost', 'lgbm', 'random_forest', 
              'extra_trees', 'decision_tree', 'ada_boost', 'stacking']
    
    result_rows = []
    
    # Iterate through each simulation/row
    for idx in range(len(df)):
        row = df.iloc[idx]
        
        # Get true weights for this simulation
        true_weights = []
        for feature_idx in range(1, n_features + 1):
            true_col = f"true_weights_feature_{feature_idx}"
            if true_col in df.columns:
                true_weights.append(row[true_col])
        
        # Calculate MAD for each model
        for model in models:
            model_weights = []
            for feature_idx in range(1, n_features + 1):
                model_col = f"{model}_feature_{feature_idx}"
                if model_col in df.columns:
                    model_weights.append(row[model_col])
            
            # Only calculate if we have data for both
            if true_weights and model_weights and len(true_weights) == len(model_weights):
                try:
                    # Calculate Mean Absolute Deviation
                    mad = np.mean(np.abs(np.array(true_weights) - np.array(model_weights)))
                    
                    result_rows.append({
                        'n_features': n_features,
                        'weight_type': weight_type,
                        'model': model,
                        'sim_idx': idx,
                        'mean_abs_deviation': mad
                    })
                except Exception as e:
                    print(f"Error calculating MAD: {str(e)}")
    
    # Create summary dataframe
    mad_df = pd.DataFrame(result_rows)
    
    # Calculate summary statistics for MADs
    summary_rows = []
    for model in models:
        model_mads = mad_df[mad_df['model'] == model]
        
        if len(model_mads) > 0:
            summary_rows.append({
                'n_features': n_features,
                'weight_type': weight_type,
                'model': model,
                'mean_mad': model_mads['mean_abs_deviation'].mean(),
                'median_mad': model_mads['mean_abs_deviation'].median(),
                'std_mad': model_mads['mean_abs_deviation'].std(),
                'min_mad': model_mads['mean_abs_deviation'].min(),
                'max_mad': model_mads['mean_abs_deviation'].max()
            })
    
    return pd.DataFrame(summary_rows)

def create_visualizations(imp_df, acc_df, meta_df, n_features, weight_type, output_path):
    """Create visualization plots for each feature count and weight type"""
    feature_dir = output_path / f"features_{n_features}_weights_{weight_type}"
    feature_dir.mkdir(exist_ok=True)
    
    # 1. Feature Importance Comparison
    try:
        models = ['true_weights', 'xgboost', 'lgbm', 'random_forest', 
                'extra_trees', 'decision_tree', 'ada_boost', 'stacking']
        
        # Prepare data for plotting
        plot_data = []
        for model in models:
            for feature_idx in range(1, n_features + 1):
                col_name = f"{model}_feature_{feature_idx}"
                if col_name in imp_df.columns:
                    for value in imp_df[col_name]:
                        plot_data.append({
                            'Model': model,
                            'Feature': f"Feature {feature_idx}",
                            'Importance': value
                        })
        
        if plot_data:
            plot_df = pd.DataFrame(plot_data)
            
            plt.figure(figsize=(15, 10))
            sns.boxplot(data=plot_df, x='Feature', y='Importance', hue='Model')
            plt.title(f'Feature Importance Distribution ({n_features} Features, {weight_type} Weights)')
            plt.legend(loc='upper right')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.tight_layout()
            plt.savefig(feature_dir / f"importance_boxplot_{n_features}_{weight_type}.png", dpi=300)
            plt.close()
    except Exception as e:
        print(f"Error creating feature importance visualization: {str(e)}")
    
    # 2. Accuracy Comparison
    if acc_df is not None:
        try:
            plt.figure(figsize=(12, 8))
            acc_models = [col for col in acc_df.columns 
                        if col not in ['scenario_id', 'n_features', 'weight_type', 
                                      'sample_size', 'n_simulations', 'Unnamed: 0']]
            sns.boxplot(data=acc_df[acc_models])
            plt.title(f'Model Accuracy Comparison ({n_features} Features, {weight_type} Weights)')
            plt.ylabel('Accuracy')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.tight_layout()
            plt.savefig(feature_dir / f"accuracy_boxplot_{n_features}_{weight_type}.png", dpi=300)
            plt.close()
        except Exception as e:
            print(f"Error creating accuracy visualization: {str(e)}")
    
    # 3. Meta Weights Comparison
    if meta_df is not None:
        try:
            plt.figure(figsize=(12, 8))
            meta_cols = [col for col in meta_df.columns 
                        if col not in ['scenario_id', 'n_features', 'weight_type', 
                                      'sample_size', 'n_simulations', 'Unnamed: 0']]
            
            # Map column indices to model names
            models = ['xgboost', 'lgbm', 'random_forest', 'extra_trees', 'decision_tree', 'ada_boost']
            rename_dict = {str(i): models[i] for i in range(len(models)) if str(i) in meta_cols}
            
            if rename_dict:
                meta_plot_df = meta_df[meta_cols].rename(columns=rename_dict)
                sns.boxplot(data=meta_plot_df)
                plt.title(f'Meta-Model Weights Distribution ({n_features} Features, {weight_type} Weights)')
                plt.ylabel('Weight')
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(feature_dir / f"meta_weights_boxplot_{n_features}_{weight_type}.png", dpi=300)
                plt.close()
        except Exception as e:
            print(f"Error creating meta weights visualization: {str(e)}")
    
    # 4. Rank Correlation Visualization
    try:
        # Calculate rank correlations for this specific dataset
        rank_corrs = calculate_rank_correlations(imp_df, n_features, weight_type)
        
        if not rank_corrs.empty:
            plt.figure(figsize=(12, 8))
            sns.barplot(data=rank_corrs, x='model', y='mean_rank_corr')
            plt.title(f'Mean Rank Correlation with True Weights ({n_features} Features, {weight_type} Weights)')
            plt.ylabel('Kendals Rank Correlation')
            plt.xlabel('Model')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(feature_dir / f"rank_correlation_{n_features}_{weight_type}.png", dpi=300)
            plt.close()
    except Exception as e:
        print(f"Error creating rank correlation visualization: {str(e)}")
    
    # 5. Mean Absolute Deviation Visualization
    try:
        # Calculate MADs for this specific dataset
        mad_stats = calculate_mean_absolute_deviations(imp_df, n_features, weight_type)
        
        if not mad_stats.empty:
            plt.figure(figsize=(12, 8))
            sns.barplot(data=mad_stats, x='model', y='mean_mad')
            plt.title(f'Mean Absolute Deviation from True Weights ({n_features} Features, {weight_type} Weights)')
            plt.ylabel('Mean Absolute Deviation')
            plt.xlabel('Model')
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(feature_dir / f"mean_abs_deviation_{n_features}_{weight_type}.png", dpi=300)
            plt.close()
    except Exception as e:
        print(f"Error creating MAD visualization: {str(e)}")

def create_feature_weight_comparison(fi_df, acc_df, meta_df, corr_df, mad_df, output_path):
    """Create comparison visualizations across feature counts and weight types for Stacking"""
    
    # Focus only on Stacking model results
    stacking_acc_df = acc_df[acc_df['model'] == 'stacking']
    stacking_corr_df = corr_df[corr_df['model'] == 'stacking']
    stacking_mad_df = mad_df[mad_df['model'] == 'stacking']
    
    # 1. Stacking Accuracy Heatmap
    try:
        # Create pivot table for stacking accuracy
        pivot_acc_data = stacking_acc_df.pivot_table(
            index='n_features',
            columns='weight_type',
            values='mean_accuracy',
            aggfunc='mean'
        )
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(pivot_acc_data, annot=True, cmap='viridis', fmt='.4f', 
                    cbar_kws={'label': 'Stacking Accuracy'})
        plt.title('Stacking Accuracy by Feature Count and Weight Type')
        plt.xlabel('Weight Type')
        plt.ylabel('Number of Features')
        plt.tight_layout()
        plt.savefig(output_path / "stacking_accuracy_heatmap.png", dpi=300)
        plt.close()
    except Exception as e:
        print(f"Error creating stacking accuracy heatmap: {str(e)}")
    
    # 2. Stacking Rank Correlation Heatmap
    if not stacking_corr_df.empty:
        try:
            # Create pivot table for stacking rank correlation
            pivot_corr_data = stacking_corr_df.pivot_table(
                index='n_features',
                columns='weight_type',
                values='mean_rank_corr',
                aggfunc='mean'
            )
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(pivot_corr_data, annot=True, cmap='coolwarm', fmt='.4f', center=0,
                        cbar_kws={'label': 'Rank Correlation'})
            plt.title('Stacking Rank Correlation by Feature Count and Weight Type')
            plt.xlabel('Weight Type')
            plt.ylabel('Number of Features')
            plt.tight_layout()
            plt.savefig(output_path / "stacking_rank_correlation_heatmap.png", dpi=300)
            plt.close()
        except Exception as e:
            print(f"Error creating stacking rank correlation heatmap: {str(e)}")
    
    # 3. Stacking Mean Absolute Deviation Heatmap
    if not stacking_mad_df.empty:
        try:
            # Create pivot table for stacking MAD
            pivot_mad_data = stacking_mad_df.pivot_table(
                index='n_features',
                columns='weight_type',
                values='mean_mad',
                aggfunc='mean'
            )
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(pivot_mad_data, annot=True, cmap='YlOrRd_r', fmt='.4f',
                        cbar_kws={'label': 'Mean Absolute Deviation'})
            plt.title('Stacking Mean Absolute Deviation by Feature Count and Weight Type')
            plt.xlabel('Weight Type')
            plt.ylabel('Number of Features')
            plt.tight_layout()
            plt.savefig(output_path / "stacking_mean_absolute_deviation_heatmap.png", dpi=300)
            plt.close()
        except Exception as e:
            print(f"Error creating stacking MAD heatmap: {str(e)}")
            
    # Generate comprehensive summary report
    generate_summary_report(fi_df, acc_df, meta_df, corr_df, mad_df, output_path)

def generate_summary_report(fi_df, acc_df, meta_df, corr_df, mad_df, output_path):
    """Generate a detailed text report summarizing all findings"""
    report_path = output_path / "feature_weight_summary_report.txt"
    
    with open(report_path, 'w') as f:
        f.write("SIMULATION STUDY SUMMARY REPORT BY FEATURE COUNT AND WEIGHT TYPE\n")
        f.write("=" * 70 + "\n\n")
        
        # Overview
        unique_features = sorted(fi_df['n_features'].unique())
        unique_weights = sorted(fi_df['weight_type'].unique())
        f.write(f"Analysis covers {len(unique_features)} feature counts: {', '.join(map(str, unique_features))}\n")
        f.write(f"Analysis covers {len(unique_weights)} weight types: {', '.join(map(str, unique_weights))}\n\n")
        
        # Part 1: Accuracy Summary
        f.write("PART 1: MODEL ACCURACY SUMMARY\n")
        f.write("-" * 50 + "\n\n")
        
        # Best model by feature count and weight type
        f.write("Best Performing Models by Feature Count and Weight Type:\n")
        for n_features in unique_features:
            for weight_type in unique_weights:
                subset = acc_df[(acc_df['n_features'] == n_features) & 
                                (acc_df['weight_type'] == weight_type)]
                
                if not subset.empty:
                    best_model = subset.loc[subset['mean_accuracy'].idxmax()]
                    f.write(f"  Features: {n_features}, Weight Type: {weight_type} - Best Model: {best_model['model']} ")
                    f.write(f"(Accuracy: {best_model['mean_accuracy']:.4f} ± {best_model['std_accuracy']:.4f})\n")
        
        f.write("\nOverall Best Performing Models (Averaged across scenarios):\n")
        model_avg = acc_df.groupby('model')['mean_accuracy'].mean().sort_values(ascending=False)
        for model, avg_acc in model_avg.items():
            f.write(f"  {model}: {avg_acc:.4f}\n")
        
        # Part 2: Feature Importance Summary
        f.write("\n\nPART 2: FEATURE IMPORTANCE SUMMARY\n")
        f.write("-" * 50 + "\n\n")
        
        # Rank correlation summary
        if corr_df is not None and not corr_df.empty:
            f.write("Feature Importance Rank Correlation with True Weights:\n")
            for n_features in unique_features:
                for weight_type in unique_weights:
                    subset = corr_df[(corr_df['n_features'] == n_features) & 
                                    (corr_df['weight_type'] == weight_type)]
                    
                    if not subset.empty:
                        f.write(f"\n  Features: {n_features}, Weight Type: {weight_type}\n")
                        
                        # Sort models by rank correlation
                        sorted_models = subset.sort_values('mean_rank_corr', ascending=False)
                        for _, row in sorted_models.iterrows():
                            f.write(f"    {row['model']}: {row['mean_rank_corr']:.4f} ")
                            f.write(f"(Range: {row['min_rank_corr']:.4f} to {row['max_rank_corr']:.4f})\n")
        
        # Mean Absolute Deviation summary
        if mad_df is not None and not mad_df.empty:
            f.write("\nMean Absolute Deviation from True Weights:\n")
            for n_features in unique_features:
                for weight_type in unique_weights:
                    subset = mad_df[(mad_df['n_features'] == n_features) & 
                                   (mad_df['weight_type'] == weight_type)]
                    
                    if not subset.empty:
                        f.write(f"\n  Features: {n_features}, Weight Type: {weight_type}\n")
                        
                        # Sort models by MAD (ascending - lower is better)
                        sorted_models = subset.sort_values('mean_mad', ascending=True)
                        for _, row in sorted_models.iterrows():
                            f.write(f"    {row['model']}: {row['mean_mad']:.4f} ")
                            f.write(f"(Range: {row['min_mad']:.4f} to {row['max_mad']:.4f})\n")
        
        # Part 3: Meta-Model Weights Summary
        if meta_df is not None and not meta_df.empty:
            f.write("\n\nPART 3: META-MODEL WEIGHTS SUMMARY\n")
            f.write("-" * 50 + "\n\n")
            
            f.write("Distribution of Meta-Model Weights by Feature Count and Weight Type:\n")
            for n_features in unique_features:
                for weight_type in unique_weights:
                    subset = meta_df[(meta_df['n_features'] == n_features) & 
                                    (meta_df['weight_type'] == weight_type)]
                    
                    if not subset.empty:
                        f.write(f"\n  Features: {n_features}, Weight Type: {weight_type}\n")
                        
                        # Sort base models by mean weight
                        sorted_models = subset.sort_values('mean_weight', ascending=False)
                        for _, row in sorted_models.iterrows():
                            f.write(f"    {row['base_model']}: {row['mean_weight']:.4f} ")
                            f.write(f"(Range: {row['min_weight']:.4f} to {row['max_weight']:.4f})\n")
        
        # Part 4: Key Findings and Recommendations
        f.write("\n\nPART 4: KEY FINDINGS AND RECOMMENDATIONS\n")
        f.write("-" * 50 + "\n\n")
        
        # Identify overall best model
        overall_best_model = acc_df.groupby('model')['mean_accuracy'].mean().idxmax()
        overall_best_acc = acc_df.groupby('model')['mean_accuracy'].mean().max()
        
        f.write(f"1. Overall Best Performing Model: {overall_best_model} (Average Accuracy: {overall_best_acc:.4f})\n")
        
        # Identify best model for feature importance ranking
        if corr_df is not None and not corr_df.empty:
            best_ranking_model = corr_df.groupby('model')['mean_rank_corr'].mean().idxmax()
            best_ranking_corr = corr_df.groupby('model')['mean_rank_corr'].mean().max()
            f.write(f"2. Best Model for Feature Importance Ranking: {best_ranking_model} ")
            f.write(f"(Average Rank Correlation: {best_ranking_corr:.4f})\n")
        
        # Best model for different feature counts
        f.write("3. Best Models by Feature Count:\n")
        for n_features in unique_features:
            subset = acc_df[acc_df['n_features'] == n_features]
            if not subset.empty:
                best_model = subset.groupby('model')['mean_accuracy'].mean().idxmax()
                best_acc = subset.groupby('model')['mean_accuracy'].mean().max()
                f.write(f"   - {n_features} Features: {best_model} (Average Accuracy: {best_acc:.4f})\n")
        
        # Best model for different weight types
        f.write("4. Best Models by Weight Type:\n")
        for weight_type in unique_weights:
            subset = acc_df[acc_df['weight_type'] == weight_type]
            if not subset.empty:
                best_model = subset.groupby('model')['mean_accuracy'].mean().idxmax()
                best_acc = subset.groupby('model')['mean_accuracy'].mean().max()
                f.write(f"   - {weight_type} Weights: {best_model} (Average Accuracy: {best_acc:.4f})\n")
        
        # Additional findings
        f.write("\nAdditional Observations:\n")
        
        # Feature count impact
        f.write("- Impact of Feature Count on Performance:\n")
        feature_impact = acc_df.groupby('n_features')['mean_accuracy'].mean()
        for n_features, mean_acc in feature_impact.items():
            f.write(f"  {n_features} Features: {mean_acc:.4f}\n")
        
        # Weight type impact
        f.write("- Impact of Weight Type on Performance:\n")
        weight_impact = acc_df.groupby('weight_type')['mean_accuracy'].mean()
        for weight_type, mean_acc in weight_impact.items():
            f.write(f"  {weight_type} Weights: {mean_acc:.4f}\n")
        
        # Recommendations
        f.write("\nRecommendations:\n")
        f.write(f"1. For optimal predictive performance, the {overall_best_model} model is recommended.\n")
        
        if corr_df is not None and not corr_df.empty:
            f.write(f"2. For feature importance interpretation, the {best_ranking_model} model provides ")
            f.write("the most reliable feature rankings.\n")
        
        f.write("3. Meta-model (stacking) performance suggests that combining models ")
        f.write("may provide more stable results across different scenarios.\n")
        
        f.write("\nReport generated on: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
    
    print(f"Summary report generated: {report_path}")

if __name__ == "__main__":
    # Replace with your master folder path
    master_folder = "G:\\UOC\\Level 4\\Research\\Outputs\\Method 2\\Classification\\simulation_results"
    output_folder = "feature_analysis_summaries_classification_02"
    
    # Run the summary analysis
    summarize_by_feature_count_and_weight(master_folder, output_folder)

Scanning G:\UOC\Level 4\Research\Outputs\Method 2\Regression\simulation_results for simulation results...
Found 112 valid scenario folders

Processing 6 scenarios with 3 features and weight type EQ...

Processing 6 scenarios with 3 features and weight type GRAD_DEC...

Processing 6 scenarios with 3 features and weight type HIGH_LOW...

Processing 6 scenarios with 3 features and weight type ONE_DOM...

Processing 6 scenarios with 3 features and weight type TWO_DOM...

Processing 6 scenarios with 5 features and weight type EQ...

Processing 6 scenarios with 5 features and weight type GRAD_DEC...

Processing 6 scenarios with 5 features and weight type HIGH_LOW...

Processing 5 scenarios with 5 features and weight type ONE_DOM...

Processing 5 scenarios with 5 features and weight type TWO_DOM...

Processing 5 scenarios with 7 features and weight type EQ...

Processing 5 scenarios with 7 features and weight type GRAD_DEC...

Processing 5 scenarios with 7 features and weight type HIGH_LOW...